In [25]:
%%capture --no-stderr
%pip install pyautogen~=0.2.0b4

In [2]:
import autogen

config_list_gpt4 = autogen.config_list_from_json("OAI_CONFIG_LIST")

In [3]:
gpt4_config = {
    "cache_seed": 9,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
    "max_tokens": 4000,
}
user_proxy = autogen.UserProxyAgent(
   name="Boss",
   system_message="""Boss. 与Product-Manager，Planner, Engineer, BashCoder, Executor 讨论当前提出的任务的方案与计划。计划和方案执行需要得到该Boss的批准。
   Executor 是理解力低下的Agent，不要试图告诉先做什么再做什么，最好是把让它做的，通过 bash 脚本的方式告诉它。
   """,
   code_execution_config=False,
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. 为当前项目拟出计划。根据Boss和Product-Manager,Engineer, BashCoder以及 Executor的反馈修改计划和方案，直到Boss批准。
该计划可能涉及一个会写代码的Engineer,一个用Bash 将Engineer 写的代码转换成可以将Engineer的代码存入本地文件的BashCoder和一个用执行bash脚本来的Executor。
Engineer 的代码需要 BashCoder 用Bash 来存储到本地，然后由 Executor 执行，所以要求 BashCoder 需要将Engineer的代码内容写成Bash脚本后再给 Executor 执行。
首先解释计划。清楚哪一步是由哪个 Agent完成""",
    llm_config=gpt4_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. 你遵循批准的计划，你是一个工程素养很高的工程师，对批准的计划需求进行编程。
    你会告诉 BashCoder 把对应的需求代码放在不同的文件夹下的不同文件中。
    用户无法修改您的代码,为了不必要的执行报错和修正，所以不要建议不完整的代码需要其他人修改或补充，回复的代码块必须是完整的，而不是部分代码或伪代码。如果代码块不打算执行，就不要使用它。
    另外程序的代码很有可能是多个代码文件组合在一起的，所以你写完一个代码文件就让BashCoder转成Bash脚本并由 Executor执行这个Bash脚本，然后再写下一个代码文件。记住务必每一次和BashCoder沟通的内容只包含一个要BashCoder转换的bash脚本任务。当所有的文件都写完后，最后你再写一个bash脚本异步的方式启动你的程序，并告诉Executer用其执行。
    每次给BashCoder 或Executor 时都只派发一个代码文件的内容
    检查Executor返回的执行结果，如果结果表明存在错误，请修复错误并再次输出代码。如果错误无法修复，或者即使在代码成功执行后任务也没有解决，请分析问题，重新审视您的假设，收集所需的其他信息，并想出不同的方法进行尝试。""",
)

pm = autogen.AssistantAgent(
    name="Product-Manager",
    system_message="Product Manager. 仔细检查其他 Agent 成员的计划、声明和代码，并提供反馈。需要确保 Boss 的需求是真的完成",
    llm_config=gpt4_config,
)


executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. 执行BashCoder将送过来的Bash脚本并报告结果。",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "site", "use_docker": "false"},
    # llm_config=gpt4_config,
)
code = autogen.UserProxyAgent(
    name="BashCoder",
    system_message="BashCoder. 接收Engineer的代码且按照Engineer说的要求，写一段Bash用来存储Engineer的代码，最后交给Executor执行。如果是要执行的代码是启动程序，就需要用 nohup 的方式启动。",
    human_input_mode="NEVER",
    code_execution_config= False,
    # code_execution_config={"last_n_messages": 3, "work_dir": "site", "use_docker": "false"},
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, planner, executor, pm, code], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)


In [4]:
user_proxy.initiate_chat(
    manager,
    message="""
写一个 Web 版的贪吃蛇游戏，需要用NodeJS 启动运行。
""",
)

Boss (to chat_manager):


写一个 Web 版的贪吃蛇游戏，需要用NodeJS 启动运行。


--------------------------------------------------------------------------------
Planner (to chat_manager):

为了完成Boss的要求，我们需要制定一个详细的计划来开发一个基于Web的贪吃蛇游戏。以下是计划的步骤和各个角色的职责：

1. **项目规划阶段**:
   - Product Manager: 定义游戏的功能需求，界面设计，以及用户体验目标。
   - Engineer: 提供技术可行性分析，估计开发时间和资源。
   - Planner: 根据上述信息，制定项目计划和时间表。

2. **开发阶段**:
   - Engineer: 负责编写游戏的前端和后端代码。前端使用HTML, CSS, JavaScript实现用户界面和游戏逻辑。后端使用Node.js创建服务器，处理游戏状态和玩家之间的通信。
   - BashCoder: 负责编写Bash脚本，用于自动化部署流程，包括将Engineer的代码存储到本地文件系统，并确保Executor可以执行这些脚本。
   - Executor: 负责运行BashCoder提供的脚本，以部署和启动Node.js服务器，从而使游戏在线上运行。

3. **测试阶段**:
   - Engineer: 进行单元测试和集成测试，确保代码的稳定性和性能。
   - Product Manager: 进行用户测试，确保游戏的用户体验符合预期。
   - Planner: 根据测试结果调整计划和时间表。

4. **部署阶段**:
   - BashCoder: 更新Bash脚本以适应生产环境的部署需求。
   - Executor: 使用更新后的Bash脚本在生产环境中部署游戏。

5. **维护阶段**:
   - Engineer: 根据用户反馈和监控数据，对游戏进行必要的维护和更新。
   - Product Manager: 监控游戏的用户反馈，提出新的功能需求。
   - Planner: 定期更新项目计划，以适应新的需求和市场变化。

在每个阶段结束时，Planner需要收集来自所有相